# 4-VSC system test case collection Notebook
# RMS vs EMT differences in Stability Analysis

## Plotting Function

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib widget

In [ ]:
def autoscale_y(ax,margin):
    """This function rescales the y-axis based on the data that is visible given the current xlim of the axis.
    ax -- a matplotlib axes object
    margin -- the fraction of the total height of the y-data to pad the upper and lower ylims"""

    def get_bottom_top(line):
        xd = line.get_xdata()
        yd = line.get_ydata()
        lo,hi = ax.get_xlim()
        y_displayed = yd[((xd>lo) & (xd<hi))]
        h = np.max(y_displayed) - np.min(y_displayed)
        bot = np.min(y_displayed)-margin*h
        top = np.max(y_displayed)+margin*h
        return bot,top

    lines = ax.get_lines()
    bot,top = np.inf, -np.inf

    for line in lines:
        new_bot, new_top = get_bottom_top(line)
        if new_bot < bot: bot = new_bot
        if new_top > top: top = new_top

    ax.set_ylim(bot,top)

In [ ]:
def plot_results(scenario, stepss_file, dynawo_file, emtp_file):

    stepss = pd.read_csv(stepss_file, sep = '\s+', comment=";", keep_default_na=False, names=('time', 'BUS_A', 'BUS_B', 'BUS_C', 'BUS_E', 'BUS_F'))
    dynawo= pd.read_csv(dynawo_file, sep = ';')
    
    emtp= pd.read_csv(emtp_file, sep = '\t', low_memory=False)
    emtp.drop(index=emtp.index[0], axis=0, inplace=True)
    emtp.rename(columns=lambda x: x.strip(), inplace=True)

    stepss_variables= ['BUS_A', 'BUS_B', 'BUS_E', 'BUS_F']
    dynawo_variables= ['NETWORK__BUS__A_TN_Upu_value', 'NETWORK__BUS__B_TN_Upu_value', 'NETWORK__BUS__E_TN_Upu_value', 'NETWORK__BUS__F_TN_Upu_value']
    emtp_variables= ['uamp_pcc_A_pu', 'uamp_pcc_B_pu', 'uamp_pcc_E_pu',	'uamp_pcc_F_pu']
    labels= ['Bus A Vpcc (p.u)', 'Bus B Vpcc (p.u)', 'Bus E Vpcc (p.u)', 'Bus F Vpcc (p.u)']
    
    plt.close('all')
    fig=plt.figure(figsize=(15, 12))
    plt.suptitle(scenario , fontsize=16)
    plt.subplots_adjust(top=0.9, hspace=0.4, wspace=0.4)
    plt.tight_layout()
    
    for n, (x, y, z, l) in enumerate(zip(stepss_variables, dynawo_variables, emtp_variables, labels)):
        ax = plt.subplot(2, 2, n + 1)
        ax.plot(stepss['time'], stepss[x], color= 'black', linestyle='-', label='Stepss')
        ax.plot(dynawo['time'], dynawo[y], color= 'tab:green', linestyle=':', label='Dynawo')
        ax.plot(pd.to_numeric(emtp['X axis'][::10]), pd.to_numeric(emtp[z][::10]), color= 'blue', linestyle=':', label='EMTP')
               
        ax.set_title(l, fontsize=14)
        ax.set_ylabel(l, fontsize=14)
        ax.tick_params(axis='both', which='major', labelsize=14)
        ax.tick_params(axis='both', which='minor', labelsize=14)
        ax.legend(fontsize=14, loc="lower right")
        ax.legend(fontsize=14, loc="upper right")
        ax.set_xlim([3, 8])

        autoscale_y(ax, 10)
        
        # if ax.get_title() in ["P (MW)", "Q (Mvar)"]:
        #     autoscale_y(ax, 0.1)
        # else:
        #     autoscale_y(ax, 0.1)

## **Stability limit as function of External network strength (short circuit power)**

### Scenario 1:
- Initial state: Operating point 1
- Event: Mild disturbance: line disconnection

In [ ]:
plot_results('Scenario 1: SCR 20000MMVA , Operating point 1, Mild disturbance: line disconnection',
             './results/Studies/SCR/stepss_OP1_LD_SC20.csv', # 20000MVA (0.005 p.u)
             './results/Studies/SCR/dynawo_OP1_LD_SC20.csv',  # 20000MVA (0.005 p.u)
            './results/Studies/SCR/emtp_OP1_LD_SC20.csv') # 20000MVA (0.005 p.u)

In [ ]:
plot_results('Scenario 1: SCR Limits , Operating point 1, Mild disturbance: line disconnection',
             './results/Studies/SCR/stepss_OP1_LD_SC4.csv', # 4000MVA (0.025 p.u)
             './results/Studies/SCR/dynawo_OP1_LD_SC4.csv', # 4000MVA (0.025 p.u)
             # './results/Studies/SCR/emtp_OP1_LD_SC15.csv') # 15000 MVA (10.666 Ohm)
             './results/Studies/SCR/emtp_OP1_LD_SC14.csv') # 14000 MVA (11.42857 Ohm)
             # './results/Studies/SCR/emtp_OP1_LD_SC13p5.csv') # 13500 MVA (11.8518518 Ohm)
             # './results/Studies/SCR/emtp_OP1_LD_SC13.csv') # 13000 MVA (12.30769 Ohm)

### Scenario 2:
- Initial state: Operating point 2
- Event: Mild disturbance: line disconnection

In [ ]:
plot_results('Scenario 2: SCR 20000MMVA , Operating point 2, Mild disturbance: line disconnection',
             './results/Studies/SCR/stepss_OP2_LD_SC20.csv', # 20000MVA (0.005p.u)
             './results/Studies/SCR/dynawo_OP2_LD_SC20.csv', # 20000MVA (0.005p.u)
             './results/Studies/SCR/emtp.csv') # 20000MVA (0.005p.u)

In [ ]:
plot_results('Scenario 2: SCR 5000MMVA , Operating point 2, Mild disturbance: line disconnection',
             './results/Studies/SCR/stepss_OP2_LD_SC5.csv', # 5000MVA (0.02p.u)
              # './results/Studies/SCR/dynawo_OP2_LD_SC10.csv', # 10000MVA (0.01p.u)
             './results/Studies/SCR/dynawo_OP2_LD_SC5.csv', # 5000MVA (0.02p.u)
             './results/Studies/SCR/emtp_OP1_LD_SC16.csv') # 5000MVA (0.02p.u)

- Visualization:
    - VSCs PCC bus Votages
    - VSCs injected active/reactive power (prior to transformer at converter terminal)
    
- Post processing